In [1]:
import pandas as pd, pyodbc

In [2]:
conn_prod = pyodbc.connect('Driver={SQL Server}; Server=10.241.109.41,20010\\OASYSHDB;Database=EPSA_Reporting;UID=epsareportes; PWD=Epsa.2020!;')
conn_dev = pyodbc.connect('Driver={SQL Server}; Server=10.241.114.12,20010\\OASYSHDB;Database=ADMS_QueryEngine;UID=Epsareportes; PWD=cmXoasys2;')


In [29]:
analog_prod = pd.read_sql_query("select name, customID,powerResource,gid from ADMS_QueryEngine.dms.AnalogSignal where name like '%_CT-%'",conn_dev, coerce_float=False)
CT_Breaker = pd.read_sql_query('''select CT.name as CurrentTransformer_Name, CT.container as BayGid, CT.gid as CurrentTransformerGid, Br.gid as BreakerGid, Br.aorGroup as BreakerAorGroup from ADMS_QueryEngine.dms.CurrentTransformer as CT
    join ADMS_QueryEngine.dms.Bay as B ON B.gid = CT.container 
    join ADMS_QueryEngine.dms.BReaker as Br on Br.container = B.gid
    union 
    select CT.name as CurrentTransformer_Name, CT.container as BayGid, CT.gid as CurrentTransformerGid, Br.gid as BreakerGid, Br.aorGroup from ADMS_QueryEngine.dms.CurrentTransformer as CT
    join ADMS_QueryEngine.dms.Bay as B ON B.gid = CT.container
    join ADMS_QueryEngine.dms.PortableCell as R on R.parentContainer = B.gid 
    join ADMS_QueryEngine.dms.BReaker as Br on Br.container = R.gid

    ''',conn_dev, coerce_float=False)

In [30]:
CT_Breaker_merged = CT_Breaker.merge(analog_prod,left_on='CurrentTransformerGid', right_on='powerResource', how='inner')[['CurrentTransformerGid','BreakerGid','BreakerAorGroup']].drop_duplicates()

In [31]:
template = """  <Script>
    <Name>{}</Name>
    <Description />
    <Resource>1734167331516776448</Resource>
    <Properties>
      <UpdateProperty>
        <PropertyID>1729382256910271497</PropertyID>
        <Operation>{}</Operation>
        <ReferencedResource />
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1152921504606850057</PropertyID>
        <Operation>{}</Operation>
        <ReferencedResource />
      </UpdateProperty>
    </Properties>
    <Where>[Power Resource] = {}</Where>
    <Action>Update</Action>
    <TimeOfCreated>16:18:58 01/06/2022</TimeOfCreated>
  </Script>"""

In [32]:
body = """<?xml version="1.0" encoding="utf-8"?>
<ArrayOfScript xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">"""
for i,x in CT_Breaker_merged.iterrows():
    body = body + '\n'+ template.format('.',x.BreakerGid, x.BreakerAorGroup,x.CurrentTransformerGid)
body = body + '\n'+ '</ArrayOfScript>'

In [33]:
with open('.\CT2Breaker.xml', 'w+') as xmlFile:
    xmlFile.write(body)
    xmlFile.close()

In [23]:
CT_Breaker_merged.to_csv('CT2Breaker.csv',index = False)

In [16]:
CT_Breaker_merged

,CurrentTransformerGid,BreakerGid,BreakerAorGroup
0,288793450659568688,288793682587818039,289075213398705518
6,288793450659568732,288793682587818095,289075213398705518
12,288793450659568731,288793682587818094,289075213398705518
18,288793450659568725,288793682587818090,289075213398705518
24,288793450659568893,288793682587818393,292171438142522582
...,...,...,...
12164,288793450659569191,288793682587818799,292171438142522533
12167,288793450659569478,288793682587819120,1311110853835097231
12172,288793450659569098,288793682587818704,292171438142522544
12174,288793450659568370,288793682587817514,289075213398705530


In [34]:
conn_inteligrid = pyodbc.connect('Driver={SQL Server}; Server=10.241.109.41,20010\\OASYSHDB;Database=EPSA_Reporting;UID=cons_intelligrid; PWD=grid#2021;')

In [35]:
pd.read_sql_query("EXEC [dbo].[sp_UbicacionPuntosScada] @p_subestacion = N'CUCUANA'",conn_inteligrid, coerce_float=False)

DatabaseError: Execution failed on sql 'EXEC [dbo].[sp_UbicacionPuntosScada] @p_subestacion = N'CUCUANA'': ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Ad hoc access to OLE DB provider 'MSDASQL' has been denied. You must access this provider through a linked server. (7415) (SQLExecDirectW)")